In [3]:
"""
read in retail data and MeetFresh menu

"""

import pandas as pd

#retail_data
retail_data= pd.read_csv("scanner_data.csv")

#MeetFresh menu
#Combine two sheets and adjust data type
drink=pd.read_excel("MeetFresh_Menu_Plano.xlsx",sheet_name='Drink')
food=pd.read_excel("MeetFresh_Menu_Plano.xlsx",sheet_name='Food')
menu=pd.concat([food, drink], axis=0, ignore_index=True)
menu=menu.fillna(0)
float_col = menu.select_dtypes(include=['float64'])
for col in float_col.columns.values:
    menu[col] = menu[col].astype('int64')   
#menu.info(verbose=True)
#menu.head()

In [4]:
"""
Map top categories in retail data to meetFresh menu categories

Mapping stored in datafram: cate_map

"""

top16cate = dict(retail_data['SKU_Category'].value_counts().head(16))
#Filter retail data by only keeping top 16 SKU Category
#retail16=retail_data.loc[retail_data['SKU_Category'].isin(cate_map['SKU_Category'].tolist())]
#retail16.groupby('SKU_Category')['SKU'].nunique()

top16menu = ['Signature Series', 'Dessert Combo', 'Milk Teas','Fresh Milk','Fluffy', 
       'Fruit Series','Shaved Ice', 'Tofu Pudding', 'Egg Waffle', 'Small Bites', 
       'Teas','Herbal Teas',  'Almond Drink',
       'Winter Melon Teas', 'Slush', 'Create Your Own']

sku_cate_map = {'SKU_Category': top16cate.keys(), 'Menu_Category': top16menu}
cate_map = pd.DataFrame(data=sku_cate_map)


In [5]:
"""
Map top SKUs in each category to meetFresh menu Item_IDs

Mapping stored in datafram: item_map

"""
i = 0
item_map = pd.DataFrame({"SKU_Category":[],"SKU":[],"Menu_Category":[],"Item_ID":[]})
while i < 16:
    #get categories
    r_cate = cate_map.iloc[i,0]
    menu_cate = cate_map.iloc[i,1]
    #filter retail data to keep 1 SKU_Category
    df = retail_data.loc[retail_data['SKU_Category'] == r_cate]
    #keep only number of SKU needed to map to meetFresh menu
    n = menu['Menu_Category'].value_counts()[menu_cate]
    topsku = dict(df['SKU'].value_counts().head(n))
    #map SKU to menu_ID
    menu_ID=menu.loc[menu['Menu_Category']==menu_cate]['Item_ID'].tolist()
    sku_map = {'SKU': topsku.keys(), 'Item_ID': menu_ID}
    #add categories in mapping table
    map1=pd.DataFrame(data=sku_map)
    map1['SKU_Category'] = [r_cate]*n
    map1['Menu_Category'] = [menu_cate]*n
    item_map = item_map.append(map1,ignore_index = True)
    i+=1 
item_map

,SKU_Category,SKU,Menu_Category,Item_ID
0,N8U,UNJKW,Signature Series,SS1
1,N8U,COWU2,Signature Series,SS2
2,N8U,M6J9W,Signature Series,SS3
3,N8U,TL4WZ,Signature Series,SS4
4,N8U,QGK3S,Signature Series,SS5
...,...,...,...,...
142,01F,BLH3I,Create Your Own,CYO1
143,01F,8XU9Y,Create Your Own,CYO2
144,01F,1IX2L,Create Your Own,CYO3
145,01F,3GTLH,Create Your Own,CYO4


In [6]:
"""
Filter retail data and only keep relevant SKUs that's mapped to MeetFresh Menu

"""
filtered_retail = retail_data.loc[retail_data['SKU'].isin(item_map['SKU'].tolist())].reset_index()
filtered_retail = filtered_retail.drop(['index','Unnamed: 0'],axis=1)
filtered_retail = filtered_retail.merge(item_map, on='SKU', how='left')
filtered_retail

,Date,Customer_ID,Transaction_ID,SKU_Category_x,SKU,Quantity,Sales_Amount,SKU_Category_y,Menu_Category,Item_ID
0,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,0H2,Fruit Series,FS1
1,02/01/2016,1253,8,0H2,9STQJ,1.0,8.25,0H2,Fruit Series,FS3
2,02/01/2016,7548,10,N8U,UNJKW,1.0,2.11,N8U,Signature Series,SS1
3,02/01/2016,6044,12,N8U,EMJ1S,1.0,3.62,N8U,Signature Series,SS6
4,02/01/2016,592,13,P42,B2IW9,1.0,9.00,P42,Fresh Milk,FM9
...,...,...,...,...,...,...,...,...,...,...
32827,04/07/2016,10764,32897,R6E,DUV2Y,1.0,8.00,R6E,Dessert Combo,DC5
32828,04/07/2016,20203,32900,J4R,LI0IX,1.0,6.25,J4R,Slush,S4
32829,04/07/2016,20203,32900,U5F,F7FQ5,3.0,7.27,U5F,Fluffy,F3
32830,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,U5F,Fluffy,F1


In [7]:
top16menu = ['Signature Series', 'Dessert Combo', 'Milk Teas','Fresh Milk','Fluffy', 
       'Fruit Series','Shaved Ice', 'Tofu Pudding', 'Egg Waffle', 'Small Bites', 
       'Teas','Herbal Teas',  'Almond Drink',
       'Winter Melon Teas', 'Slush', 'Create Your Own']

# assumed category:
# popular: 'Signature Series', ''Dessert Combo', 'Milk Teas'
# ice: 'Fresh Milk', 'Fluffy', 'Fruit Series', 'Shaved Ice', 'Slush', 'Winter Melon Teas'
# hot: 'Tofu Pudding', 'Herbal Teas', 'Teas', 'Almond Drink'
# sweet: 'Egg Waffle', 'Small Bites'
# customized: 'Create Your Own'
assumed_categories = {
    'pop' : ['Signature Series', 'Dessert Combo', 'Milk Teas'],
    'ice' : ['Fresh Milk', 'Fluffy', 'Fruit Series', 'Shaved Ice', 'Slush', 'Winter Melon Teas'],
    'hot' : ['Tofu Pudding', 'Herbal Teas', 'Teas', 'Almond Drink'],
    'sweet' : ['Egg Waffle', 'Small Bites'],
    'cus' : ['Create Your Own'],
}

# assume zip code has uniqe distribution
zip_code = [str(70560+i) for i in range(20)]
sex = ['male', 'female']
weather = ['sunny', 'rainy', 'cloudy', 'windy']

In [8]:
import random

# sex preference
# key: [male_prob, female_prob]
sex_pref = {
    'pop': [0.5, 0,5],
    'ice': [0.8, 0.2],
    'hot': [0.7, 0.3],
    'sweet':[0.6, 0.4],
    'cus': [0.4, 0.6] 
}

filtered_retail['Sex'] = sex[-1]
filtered_retail['Weather'] = weather[-1]
filtered_retail['Zip_cd'] = zip_code[-1]


random.seed(5)
for k, v in assumed_categories.items():
    prob = sex_pref[k]
    idx = list(filtered_retail[filtered_retail['Menu_Category'].isin(v)].index)
    random.shuffle(idx)

    break_pt = int(len(idx)*prob[0])
    print(break_pt)
    filtered_retail.iloc[idx[break_pt:], -3] = 'male'


# key: [sunny_prob, rainny_prob, cloudy_prob, 'windy_prob']
weather_pref = {
    'pop': [0.3, 0.2, 0.2, 0.3],
    'ice': [0.7, 0.1, 0.1, 0.1],
    'hot': [0.2, 0.5, 0.2, 0.1],
    'sweet':[0.3, 0.3, 0.1, 0.4],
    'cus': [0.2, 0.2, 0.4, 0.4] 
}


random.seed(10)
for k, v in assumed_categories.items():
  prob = weather_pref[k]
  idx = list(filtered_retail[filtered_retail['Menu_Category'].isin(v)].index)
  random.shuffle(idx)

  accu_prob = 0
  break_pt = []
  for i in range(len(weather_pref[k])-1):
    accu_prob += weather_pref[k][i]
    break_pt.append(int(len(idx)*accu_prob))
  print(break_pt)
  
  prev = 0
  for j in range(len(break_pt)):
    print(prev, break_pt[j])
    filtered_retail.iloc[idx[prev:break_pt[j]], -2] = weather[j]
    prev = break_pt[j]

# key: [str(70560+i) for i in range(20)]
zip_pref = {
    'pop': [0.1] *5 + [0.05] *5 + [0.025] *10,
    'ice': [0.02] *8 + [0.08] *6 + [0.06] *6,
    'hot': [0.03] *2 + [0.11] *8 + [0.006] *10,
    'sweet':[0.06] *12 + [0.01] *4 + [0.06] *4,
    'cus': [0.08] *5 + [0.03] *7 + [0.04875] *8 
}


random.seed(15)
for k, v in assumed_categories.items():
  prob = zip_pref[k]
  idx = list(filtered_retail[filtered_retail['Menu_Category'].isin(v)].index)
  random.shuffle(idx)

  accu_prob = 0
  break_pt = []
  for i in range(len(zip_pref[k])-1):
    accu_prob += zip_pref[k][i]
    break_pt.append(int(len(idx)*accu_prob))
  print(break_pt)
  
  prev = 0
  for j in range(len(break_pt)):
    print(prev, break_pt[j])
    filtered_retail.iloc[idx[prev:break_pt[j]], -1] = zip_code[j]
    prev = break_pt[j]


5050
10075
3929
2205
339
[3030, 5050, 7070]
0 3030
3030 5050
5050 7070
[8815, 10075, 11334]
0 8815
8815 10075
10075 11334
[1122, 3929, 5052]
0 1122
1122 3929
3929 5052
[1102, 2205, 2573]
0 1102
1102 2205
2205 2573
[169, 339, 678]
0 169
169 339
339 678
[1010, 2020, 3030, 4040, 5050, 5555, 6060, 6565, 7070, 7575, 7827, 8080, 8332, 8585, 8837, 9090, 9342, 9595, 9847]
0 1010
1010 2020
2020 3030
3030 4040
4040 5050
5050 5555
5555 6060
6060 6565
6565 7070
7070 7575
7575 7827
7827 8080
8080 8332
8332 8585
8585 8837
8837 9090
9090 9342
9342 9595
9595 9847
[251, 503, 755, 1007, 1259, 1511, 1763, 2015, 3022, 4030, 5037, 6045, 7052, 8060, 8815, 9571, 10327, 11082, 11838]
0 251
251 503
503 755
755 1007
1007 1259
1259 1511
1511 1763
1763 2015
2015 3022
3022 4030
4030 5037
5037 6045
6045 7052
7052 8060
8060 8815
8815 9571
9571 10327
10327 11082
11082 11838
[168, 336, 954, 1571, 2189, 2806, 3424, 4042, 4659, 5277, 5310, 5344, 5378, 5411, 5445, 5479, 5512, 5546, 5580]
0 168
168 336
336 954
954 1571
15

In [9]:
filtered_retail['Sex'].value_counts()

female    21598
male      11234
Name: Sex, dtype: int64

In [10]:
filtered_retail['Weather'].value_counts()

sunny     14238
rainy      7360
windy      6125
cloudy     5109
Name: Weather, dtype: int64

In [11]:
filtered_retail['Zip_cd'].value_counts()

70569    2377
70568    2375
70563    2168
70562    2168
70564    2168
70561    1719
70560    1716
70566    1621
70567    1621
70565    1620
70571    1541
70570    1538
70573    1372
70572    1371
70579    1306
70577    1304
70578    1304
70576    1303
70574    1120
70575    1120
Name: Zip_cd, dtype: int64

In [12]:
# assume the zip_cd, weather, sex are independent
filtered_retail

,Date,Customer_ID,Transaction_ID,SKU_Category_x,SKU,Quantity,Sales_Amount,SKU_Category_y,Menu_Category,Item_ID,Sex,Weather,Zip_cd
0,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,0H2,Fruit Series,FS1,male,windy,70573
1,02/01/2016,1253,8,0H2,9STQJ,1.0,8.25,0H2,Fruit Series,FS3,female,sunny,70577
2,02/01/2016,7548,10,N8U,UNJKW,1.0,2.11,N8U,Signature Series,SS1,male,sunny,70562
3,02/01/2016,6044,12,N8U,EMJ1S,1.0,3.62,N8U,Signature Series,SS6,male,sunny,70566
4,02/01/2016,592,13,P42,B2IW9,1.0,9.00,P42,Fresh Milk,FM9,male,sunny,70571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,04/07/2016,10764,32897,R6E,DUV2Y,1.0,8.00,R6E,Dessert Combo,DC5,female,rainy,70565
32828,04/07/2016,20203,32900,J4R,LI0IX,1.0,6.25,J4R,Slush,S4,male,sunny,70578
32829,04/07/2016,20203,32900,U5F,F7FQ5,3.0,7.27,U5F,Fluffy,F3,female,cloudy,70578
32830,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,U5F,Fluffy,F1,female,sunny,70579


In [17]:
from google.colab import files
filtered_retail.to_csv('filtered_retail.csv')
files.download('filtered_retail.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>